In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)

In [ ]:
# !open file:///var/folders/6_/7xx0tpq16h9cn40mq4w5gjk80000gn/T/e4cd5bf71ff3484b8a0e9d7b0de2fad1.sqlite


In [ ]:
node = sy.orchestra.launch(name="blue-book", port="auto", dev_mode=True, reset=True)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
# start the third party service with openapi.json endpoint

In [ ]:
# import gevent
# from gevent.subprocess import Popen, PIPE

# def run_command(cmd):
#     process = Popen(cmd, stdout=PIPE, stderr=PIPE, shell=True)
#     for line in process.stdout:
#         print(line.strip().decode())

# cmd = "cd ../../../packages/grid/openapi/fastapi && uvicorn main:app --reload"
# greenlet = gevent.spawn(run_command, cmd)

In [ ]:
url = "http://127.0.0.1:8000/openapi.json"

In [ ]:
!curl $url

In [ ]:
result = domain_client.api.services.bridge.add(url=url)
result

In [ ]:
# auth required, will fail without login
result = domain_client.api.services.blue_book.get_me()
result

In [ ]:
result = domain_client.api.services.bridge.authenticate(token="letmein")
result

In [ ]:
result = domain_client.api.services.blue_book.get_me()
result

In [ ]:
result = domain_client.api.services.blue_book.get_all()
result

In [ ]:
result = domain_client.api.services.blue_book.get_model(model_id=7)
result

In [ ]:
model = domain_client.api.types.ResearchModel(name="David")
model

In [ ]:
result = domain_client.api.services.blue_book.set_model(model_id=8, researchmodel=model)
result

In [ ]:
result = domain_client.api.services.blue_book.get_all()
result

In [ ]:
assert len(result) == 2

In [ ]:
domain_client.api.services.blue_book.get_all_compute()

In [ ]:
resource = domain_client.api.services.blue_book.get_compute_config(compute_name="azure_cpu")
resource

In [ ]:
@sy.api_pre_hook(path="blue_book.get_compute_config")
def get_compute_config_pre_hook(context, kwargs):
    print("context.role", context.role)
    print("context.session", context.session)
    print("kwargs", kwargs)
    return kwargs

In [ ]:
get_compute_config_pre_hook.id

In [ ]:
response = domain_client.api.services.bridge.set_wrapper(wrapper=get_compute_config_pre_hook)
response

In [ ]:
@sy.api_post_hook(path="blue_book.get_compute_config")
def get_compute_config_post_hook(context, result):
    context.session["compute"] = result
    return result

In [ ]:
get_compute_config_post_hook.id

In [ ]:
response = domain_client.api.services.bridge.set_wrapper(wrapper=get_compute_config_post_hook)
response

In [ ]:
resource = domain_client.api.services.bridge.get_wrappers(path="blue_book.get_compute_config")
resource

In [ ]:
session = domain_client.api.services.bridge.session()
print(session.kv_store, session.authentication)

In [ ]:
resource = domain_client.api.services.blue_book.get_compute_config(compute_name="azure_cpu")
resource

In [ ]:
session = domain_client.api.services.bridge.session()
print(session.kv_store, session.authentication)

In [ ]:
assert len(session.kv_store) == 1

In [ ]:
if node.node_type.value == "python":
    node.land()